<img src='img/logo.png' />

<img src='img/title.png'>

<img src='img/py3k.png'>

# Table of Contents
* [Exercise (classes and data structures)](#Exercise-%28classes-and-data-structures%29)
	* [Classes](#Classes)
	* [Reading files](#Reading-files)
	* [Teams](#Teams)
	* [Team Revenue](#Team-Revenue)

# Exercise (classes and data structures)

In our fictional company we have information about employees (name, date hired, etc.) and transaction data for the widgets and frobniz we sell. In this exercise create classes and functions to help the company understand the current organizational structure and generate revenue reports.

1. Design an `Employee` class based on the data provided in `data/employees.csv`
1. Include the transactions in `data/transactions.csv` in `Employee` objects.
1. Create functions to perform the following tasks
    * Create a collection of teams based on Subordinates of Manager Employees
    * Generate a report of total revenue and items (widgets and frobniz) sold by each team.

## Classes

In [ ]:
from collections import namedtuple
Transaction=namedtuple('Transaction', 'id item amount price')

class Employee(object):
    def __init__(self,name=None, idnumber=None, position=None, manager=None, date=None, transactions=None):
        self.Name = name
        self.Id = idnumber
        self.Position = position
        self.DateHired = date
        self.Manager = manager

        self.Transactions=[]
        if transactions is not None:
            self.Transactions.extend(transactions)
    
    def __hash__(self):
        return hash(self.Name)
    
    def setManager(self,manager):
        self.Manager=manager
    
    def addTransaction(self,transaction):
        self.Transactions.append(transaction)
    
    def countItems(self):
        return sum([trans.amount for trans in self.Transactions])
    
    def totalRevenue(self):
        return sum([trans.price * trans.amount for trans in self.Transactions])

## Reading files

In [ ]:
import csv
import datetime

def csvToList(csvfile):
    lines=[]
    # read csv
    with open(csvfile) as f:
        reader=csv.reader(f)
        for row in reader:
            lines.append(row)
    return lines

def readEmployees(csvfile):
    employees=[]
    lines=csvToList(csvfile)
    # make employee objects
    for line in lines[1:]:
        date=datetime.date(*[int(i) for i in line[4].split('-')])
        employees.append(Employee(name=line[0],idnumber=int(line[1]),position=line[2],manager=int(line[3]),date=date))
    # replace Manager integers with employee objects
    [emp.setManager(emp2) for emp in employees for emp2 in employees if emp.Manager == emp2.Id]
    # The owner is her own manager
    employees[0].setManager(employees[0])
    return employees

def readTransactions(csvfile,employees):
    lines=csvToList(csvfile)
    for line in lines[1:]:
        tId=int(line[0])
        eId=int(line[1])
        iType=line[2]
        amount=int(line[3])
        uPrice=float(line[4])
        transaction = Transaction(tId,iType,amount,uPrice)
        [emp.addTransaction(transaction) for emp in employees if emp.Id == eId]
    

employees=readEmployees('data/employees.csv')
readTransactions('data/transactions.csv',employees)

## Teams

In [ ]:
from collections import defaultdict
def createTeams(employees):
    teams = defaultdict(list)
    
    for employee in employees:
        teams[employee.Manager].append(employee)
    
    return teams

In [ ]:
teams=createTeams(employees)
for manager,emp in teams.items():
    print('Manager: %s'% manager.Name)
    for i in emp:
        print('  %s' % i.Name)

## Team Revenue

In [ ]:
def teamRevenue(teams):
    report=namedtuple('report', 'manager revenue items')
    teamrevenue=[]
    for manager,employees in teams.items():
        revenue=sum([emp.totalRevenue() for emp in employees]) + manager.totalRevenue()
        items = sum([emp.countItems() for emp in employees]) + manager.countItems()
        teamrevenue.append(report(manager,revenue,items))
    return teamrevenue

In [ ]:
reports=teamRevenue(teams)
for i in reports:
    print("%s's team:" % i.manager.Name)
    print("  %d items" % i.items)
    print("  $%.2f" % i.revenue)

<img src='img/copyright.png'>